## Загрузка данных

### Библиотеки:

In [1]:
import numpy as np
import pandas as pd

from pymorphy2 import MorphAnalyzer

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

### Файлы

#### Список фраз

Формат xlsx:

In [2]:
df = pd.read_excel('keywords.xlsx')

Формат csv:

In [3]:
#df = pd.read_csv('keywords.csv')

Из txt файла:

In [4]:
#keywords = [phrase.rstrip() for phrase in open('keywords.txt')]

In [5]:
#df = pd.DataFrame(keywords, columns=['Фраза'])

#### Загрузка стоп-слов

In [6]:
list_of_stop_words = pd.read_excel('spisok_stop_slov.xlsx', names=['stop_slova'])

#### Список предлогов

In [7]:
list_of_pretext = pd.read_excel('spisok_predlogov.xlsx', names=['predlogi'])

#### Список городов

In [8]:
list_of_cities = pd.read_excel('spisok_gorodov.xlsx', names=['goroda'])

#### Список вопросительных слов

In [9]:
list_of_questions = pd.read_excel('spisok_vopros_slov.xlsx', names=['vopros'])

#### Список "коммерческих" слов

In [10]:
list_of_commerce_words = pd.read_excel('spisok_kommercheskih_slov.xlsx', names=['komm'])

## Удаление стоп слов

Разделение фразы на слова:

In [11]:
df['Фраза_по_словам'] = df['Фраза'].str.split()

In [12]:
df

,Фраза,Базовая частота,Частота **,Частота *!*,Фраза_по_словам
0,ремонт гидроцилиндров,794,158,153,"[ремонт, гидроцилиндров]"
1,ремонт гидроцилиндров в спб,173,111,111,"[ремонт, гидроцилиндров, в, спб]"
2,вира ремонт гидроцилиндров,107,27,27,"[вира, ремонт, гидроцилиндров]"
3,вира ремонт гидроцилиндров спб,77,55,55,"[вира, ремонт, гидроцилиндров, спб]"
4,ремонт штока гидроцилиндра,24,7,6,"[ремонт, штока, гидроцилиндра]"
...,...,...,...,...,...
3236,справочник по термообработке металлов,1,0,0,"[справочник, по, термообработке, металлов]"
3237,лазерная термообработка металлов,1,0,0,"[лазерная, термообработка, металлов]"
3238,методы термообработки металлов,1,0,0,"[методы, термообработки, металлов]"
3239,термообработка металла 5,1,0,0,"[термообработка, металла, 5]"


Поиск совпадений. Если фраза содержит стоп-слово, значение меняется на 'stop':

In [13]:
def find_stop_words(splitted_phrase):
    for word in list_of_stop_words['stop_slova']:
        if word in splitted_phrase:
            return 'stop'

К столбцу "Фраза_по_словам" применяется функция, результат записывается в новый столбец "Содержит_стоп_слово":

In [14]:
df['Содержит_стоп_слово'] = df['Фраза_по_словам'].apply(find_stop_words)

Отбрасывание строк, которые содержат фразу со стоп-словами:

In [15]:
df.drop(df[df['Содержит_стоп_слово'] == 'stop'].index, inplace=True)

Удаление столбцов, который использовались для стоп-слов:

In [16]:
df.drop(columns = ['Содержит_стоп_слово', 'Фраза_по_словам'], inplace=True)

## Удаление фраз с низкой частотностью

In [17]:
#df.drop(df[df['Частота **'] == 0].index, inplace=True)

## Лемматизация ключевых слов:

Запись экземпляра класса в переменную m:

In [18]:
m = MorphAnalyzer()

Лемматизация через PyMorphy и добавление результатов в новый столбец:

In [19]:
df['Леммы'] = [' '.join([m.parse(word)[0].normal_form for word in x.split()]) for x in df['Фраза']]

In [20]:
df

,Фраза,Базовая частота,Частота **,Частота *!*,Леммы
0,ремонт гидроцилиндров,794,158,153,ремонт гидроцилиндр
1,ремонт гидроцилиндров в спб,173,111,111,ремонт гидроцилиндр в спб
2,вира ремонт гидроцилиндров,107,27,27,вира ремонт гидроцилиндр
3,вира ремонт гидроцилиндров спб,77,55,55,вира ремонт гидроцилиндр спб
4,ремонт штока гидроцилиндра,24,7,6,ремонт шток гидроцилиндр
...,...,...,...,...,...
3234,печь для термообработки металла цена,1,0,0,печь для термообработка металл цена
3235,термообработка металлов презентация,1,0,0,термообработка металл презентация
3236,справочник по термообработке металлов,1,0,0,справочник по термообработка металл
3237,лазерная термообработка металлов,1,0,0,лазерный термообработка металл


## Векторизация

### Создание экземпляра класса TfidfVectorizer(min_df=1)

In [21]:
tvidf_v = TfidfVectorizer(min_df=1)

### Обучение векторизатора леммам и преобразование набора данных

In [22]:
vectorized_lemmas = tvidf_v.fit_transform(df['Леммы'])

Чтобы посмотреть, как выглядят векторизованные фразы:

In [23]:
#pd.DataFrame(data=vectorized_lemmas.toarray(), columns=tvidf_v.get_feature_names(), index=df['Леммы'])

## Кластеризация

Настройка KMeans:

In [24]:
km_clustering = KMeans(
    n_clusters=int(np.round(np.divide(len(df['Леммы']), 5))),
    init='k-means++',
    n_init=10,
    max_iter=300,
    tol=0.0001
                    )

Кластеризация и запись кластеров в столбец:

In [25]:
df['Кластер'] = km_clustering.fit_predict(vectorized_lemmas)

Сортировка по возрастанию номера кластера:

In [26]:
df.sort_values(by='Кластер', inplace=True)

In [27]:
df

,Фраза,Базовая частота,Частота **,Частота *!*,Леммы,Кластер
926,сварочные работы за тонну,99,1,0,сварочный работа за тонна,0
1209,сварочные работы за тонну металлоконструкций,19,1,1,сварочный работа за тонна металлоконструкция,0
1105,стоимость сварочно монтажных работ трубопровод...,29,29,29,стоимость сварочный монтажный работа трубопров...,0
1199,цена сварочных работ за тонну металла,20,20,20,цена сварочный работа за тонна металл,0
961,стоимость сварочных работ за тонну,71,28,28,стоимость сварочный работа за тонна,0
...,...,...,...,...,...,...
3239,термообработка металла 5,1,0,0,термообработка металл 5,613
618,машиностроение и металлообработка в китае,2,0,0,машиностроение и металлообработка в китай,614
459,металлообработка в китае,5,2,2,металлообработка в китай,614
640,ищу исполнителя по металлообработке,2,0,0,искать исполнитель по металлообработка,615


### Вывод

Кластеризация завершена, каждой фразе присвоен номер кластера. Следующие шаги в разделе постобработка не являются обязательными. Кластеризованный датафрейм можно экспортировать:

In [28]:
#writer_kernel = pd.ExcelWriter('ready_table.xlsx', engine='xlsxwriter')
#df.to_excel(writer_kernel)
#writer_kernel.save()

## Постобработка

### Присвоим название каждому кластеру

Копирование датафрейма:

In [29]:
df2 = df.copy()

Создание нового столбца "Название кластера" и запись в него разбитых на слова фраз из столбца "Леммы":

In [30]:
df2['Название_кластера'] = df2['Леммы'].str.split()

In [31]:
df2

,Фраза,Базовая частота,Частота **,Частота *!*,Леммы,Кластер,Название_кластера
926,сварочные работы за тонну,99,1,0,сварочный работа за тонна,0,"[сварочный, работа, за, тонна]"
1209,сварочные работы за тонну металлоконструкций,19,1,1,сварочный работа за тонна металлоконструкция,0,"[сварочный, работа, за, тонна, металлоконструк..."
1105,стоимость сварочно монтажных работ трубопровод...,29,29,29,стоимость сварочный монтажный работа трубопров...,0,"[стоимость, сварочный, монтажный, работа, труб..."
1199,цена сварочных работ за тонну металла,20,20,20,цена сварочный работа за тонна металл,0,"[цена, сварочный, работа, за, тонна, металл]"
961,стоимость сварочных работ за тонну,71,28,28,стоимость сварочный работа за тонна,0,"[стоимость, сварочный, работа, за, тонна]"
...,...,...,...,...,...,...,...
3239,термообработка металла 5,1,0,0,термообработка металл 5,613,"[термообработка, металл, 5]"
618,машиностроение и металлообработка в китае,2,0,0,машиностроение и металлообработка в китай,614,"[машиностроение, и, металлообработка, в, китай]"
459,металлообработка в китае,5,2,2,металлообработка в китай,614,"[металлообработка, в, китай]"
640,ищу исполнителя по металлообработке,2,0,0,искать исполнитель по металлообработка,615,"[искать, исполнитель, по, металлообработка]"


Название каждого кластера будет состоять из всех слов, которые встречаются во фразах во всем кластере.

Сводная таблица со сложением всех фраз из кластера:

In [32]:
df_cluster_with_name = df2.pivot_table(index=['Кластер'],
                          values='Название_кластера',
                          aggfunc='sum').reset_index()

Печать несколькольких строк:

In [33]:
df_cluster_with_name

,Кластер,Название_кластера
0,0,"[сварочный, работа, за, тонна, сварочный, рабо..."
1,1,"[металлообработка, фрезерный, работа, работа, ..."
2,2,"[прайс, на, сварочный, работа, отопление, прай..."
3,3,"[токарный, работа, по, дерево, для, начинающий..."
4,4,"[сварочный, работа, метр, погонный, метр, свар..."
...,...,...
611,611,"[вид, работа, который, выполняться, на, фрезер..."
612,612,"[металлопрокат, металлообработка]"
613,613,"[справочник, по, термообработка, металл, термо..."
614,614,"[машиностроение, и, металлообработка, в, китай..."


Удаление дубликатов слов в каждой строке столбца "Название кластера".<br>
Чтобы удалить дубликаты слов, превратим list в set и снова в list:

In [34]:
df_cluster_with_name['Название_кластера'] = df_cluster_with_name['Название_кластера'].apply(lambda x: list(set(x)))

In [35]:
df_cluster_with_name

,Кластер,Название_кластера
0,0,"[за, металлоконструкция, тонна, сварочный, в, ..."
1,1,"[машинка, фрезерный, копировальный, 5, работа,..."
2,2,"[на, прайс, отопление, сварочный, работа, лист]"
3,3,"[по, на, работа, начинающий, для, сварочный, ф..."
4,4,"[метр, погонный, сварочный, работа, стоимость,..."
...,...,...
611,611,"[выполняться, на, при, токарный, который, выпо..."
612,612,"[металлопрокат, металлообработка]"
613,613,"[по, нагрев, температура, старение, для, спб, ..."
614,614,"[машиностроение, китай, в, и, металлообработка]"


Удаление предлогов из оставшихся в списках слов:

In [36]:
for pretext in list_of_pretext['predlogi']:
    for cluster_name in df_cluster_with_name['Название_кластера']:
        if pretext in cluster_name:
            cluster_name.remove(pretext)

In [37]:
df_cluster_with_name

,Кластер,Название_кластера
0,0,"[металлоконструкция, тонна, сварочный, металл,..."
1,1,"[машинка, фрезерный, копировальный, 5, работа,..."
2,2,"[прайс, отопление, сварочный, работа, лист]"
3,3,"[по, работа, начинающий, сварочный, фрезерный,..."
4,4,"[метр, погонный, сварочный, работа, стоимость,..."
...,...,...
611,611,"[выполняться, токарный, который, выполнять, ст..."
612,612,"[металлопрокат, металлообработка]"
613,613,"[по, нагрев, температура, старение, спб, 5, це..."
614,614,"[машиностроение, китай, металлообработка]"


Удаление квадратных скобок:

In [38]:
def extract_words(cluster_name):
        return ', '.join(cluster_name)

In [39]:
df_cluster_with_name['Название_кластера'] = df_cluster_with_name['Название_кластера'].apply(extract_words)

In [40]:
df_cluster_with_name

,Кластер,Название_кластера
0,0,"металлоконструкция, тонна, сварочный, металл, ..."
1,1,"машинка, фрезерный, копировальный, 5, работа, ..."
2,2,"прайс, отопление, сварочный, работа, лист"
3,3,"по, работа, начинающий, сварочный, фрезерный, ..."
4,4,"метр, погонный, сварочный, работа, стоимость, ..."
...,...,...
611,611,"выполняться, токарный, который, выполнять, ста..."
612,612,"металлопрокат, металлообработка"
613,613,"по, нагрев, температура, старение, спб, 5, цен..."
614,614,"машиностроение, китай, металлообработка"


### Слияние таблицы с название кластера и основной таблицы

Объединение сводной таблицы df_cluster_with_name, в которой названия кластера строятся из всех слов с общей таблицей df. Каждому номеру кластера в общей таблице присвоится название в столбце Название_кластера:

In [41]:
df = df.merge(df_cluster_with_name, on='Кластер')

In [42]:
df

,Фраза,Базовая частота,Частота **,Частота *!*,Леммы,Кластер,Название_кластера
0,сварочные работы за тонну,99,1,0,сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ..."
1,сварочные работы за тонну металлоконструкций,19,1,1,сварочный работа за тонна металлоконструкция,0,"металлоконструкция, тонна, сварочный, металл, ..."
2,стоимость сварочно монтажных работ трубопровод...,29,29,29,стоимость сварочный монтажный работа трубопров...,0,"металлоконструкция, тонна, сварочный, металл, ..."
3,цена сварочных работ за тонну металла,20,20,20,цена сварочный работа за тонна металл,0,"металлоконструкция, тонна, сварочный, металл, ..."
4,стоимость сварочных работ за тонну,71,28,28,стоимость сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ..."
...,...,...,...,...,...,...,...
3075,термообработка металла 5,1,0,0,термообработка металл 5,613,"по, нагрев, температура, старение, спб, 5, цен..."
3076,машиностроение и металлообработка в китае,2,0,0,машиностроение и металлообработка в китай,614,"машиностроение, китай, металлообработка"
3077,металлообработка в китае,5,2,2,металлообработка в китай,614,"машиностроение, китай, металлообработка"
3078,ищу исполнителя по металлообработке,2,0,0,искать исполнитель по металлообработка,615,"по, исполнитель, искать, металлообработка"


### Посчитаем длину фраз

Разбиваем фразу на слова:

In [43]:
df['Длина_фразы_без_предлогов'] = df['Фраза'].str.split()

Удаление предлогов:

In [44]:
for pretext in list_of_pretext['predlogi']:
    for splitted_phrase in df['Длина_фразы_без_предлогов']:
        if pretext in splitted_phrase:
            splitted_phrase.remove(pretext)

In [45]:
df

,Фраза,Базовая частота,Частота **,Частота *!*,Леммы,Кластер,Название_кластера,Длина_фразы_без_предлогов
0,сварочные работы за тонну,99,1,0,сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ...","[сварочные, работы, тонну]"
1,сварочные работы за тонну металлоконструкций,19,1,1,сварочный работа за тонна металлоконструкция,0,"металлоконструкция, тонна, сварочный, металл, ...","[сварочные, работы, тонну, металлоконструкций]"
2,стоимость сварочно монтажных работ трубопровод...,29,29,29,стоимость сварочный монтажный работа трубопров...,0,"металлоконструкция, тонна, сварочный, металл, ...","[стоимость, сварочно, монтажных, работ, трубоп..."
3,цена сварочных работ за тонну металла,20,20,20,цена сварочный работа за тонна металл,0,"металлоконструкция, тонна, сварочный, металл, ...","[цена, сварочных, работ, тонну, металла]"
4,стоимость сварочных работ за тонну,71,28,28,стоимость сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ...","[стоимость, сварочных, работ, тонну]"
...,...,...,...,...,...,...,...,...
3075,термообработка металла 5,1,0,0,термообработка металл 5,613,"по, нагрев, температура, старение, спб, 5, цен...","[термообработка, металла, 5]"
3076,машиностроение и металлообработка в китае,2,0,0,машиностроение и металлообработка в китай,614,"машиностроение, китай, металлообработка","[машиностроение, металлообработка, китае]"
3077,металлообработка в китае,5,2,2,металлообработка в китай,614,"машиностроение, китай, металлообработка","[металлообработка, китае]"
3078,ищу исполнителя по металлообработке,2,0,0,искать исполнитель по металлообработка,615,"по, исполнитель, искать, металлообработка","[ищу, исполнителя, по, металлообработке]"


Замена всех списков со словами на длину этих списков:

In [46]:
df['Длина_фразы_без_предлогов'] = df['Длина_фразы_без_предлогов'].apply(lambda x: len(x))

In [47]:
df

,Фраза,Базовая частота,Частота **,Частота *!*,Леммы,Кластер,Название_кластера,Длина_фразы_без_предлогов
0,сварочные работы за тонну,99,1,0,сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ...",3
1,сварочные работы за тонну металлоконструкций,19,1,1,сварочный работа за тонна металлоконструкция,0,"металлоконструкция, тонна, сварочный, металл, ...",4
2,стоимость сварочно монтажных работ трубопровод...,29,29,29,стоимость сварочный монтажный работа трубопров...,0,"металлоконструкция, тонна, сварочный, металл, ...",6
3,цена сварочных работ за тонну металла,20,20,20,цена сварочный работа за тонна металл,0,"металлоконструкция, тонна, сварочный, металл, ...",5
4,стоимость сварочных работ за тонну,71,28,28,стоимость сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ...",4
...,...,...,...,...,...,...,...,...
3075,термообработка металла 5,1,0,0,термообработка металл 5,613,"по, нагрев, температура, старение, спб, 5, цен...",3
3076,машиностроение и металлообработка в китае,2,0,0,машиностроение и металлообработка в китай,614,"машиностроение, китай, металлообработка",3
3077,металлообработка в китае,5,2,2,металлообработка в китай,614,"машиностроение, китай, металлообработка",2
3078,ищу исполнителя по металлообработке,2,0,0,искать исполнитель по металлообработка,615,"по, исполнитель, искать, металлообработка",4


### Определим город, если он встречается во фразе

In [48]:
df['Город'] = df['Леммы'].str.split()

In [49]:
def find_city(splitted_lemmas):
    for city in list_of_cities['goroda'].str.lower():
        if city in splitted_lemmas:
            return city

In [50]:
df['Город'] = df['Город'].apply(find_city)

In [51]:
df['Город'].fillna(value='-', inplace=True)

In [52]:
df.head(30)

,Фраза,Базовая частота,Частота **,Частота *!*,Леммы,Кластер,Название_кластера,Длина_фразы_без_предлогов,Город
0,сварочные работы за тонну,99,1,0,сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ...",3,-
1,сварочные работы за тонну металлоконструкций,19,1,1,сварочный работа за тонна металлоконструкция,0,"металлоконструкция, тонна, сварочный, металл, ...",4,-
2,стоимость сварочно монтажных работ трубопровод...,29,29,29,стоимость сварочный монтажный работа трубопров...,0,"металлоконструкция, тонна, сварочный, металл, ...",6,-
3,цена сварочных работ за тонну металла,20,20,20,цена сварочный работа за тонна металл,0,"металлоконструкция, тонна, сварочный, металл, ...",5,-
4,стоимость сварочных работ за тонну,71,28,28,стоимость сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ...",4,-
5,стоимость сварочных работ за тонну металлоконс...,18,17,17,стоимость сварочный работа за тонна металлокон...,0,"металлоконструкция, тонна, сварочный, металл, ...",5,-
6,сварочные работы цена за тонну,26,2,2,сварочный работа цена за тонна,0,"металлоконструкция, тонна, сварочный, металл, ...",4,-
7,стоимость сварочных работ за тонну металла,25,25,25,стоимость сварочный работа за тонна металл,0,"металлоконструкция, тонна, сварочный, металл, ...",5,-
8,металлообработка фрезерные работы,4,0,0,металлообработка фрезерный работа,1,"машинка, фрезерный, копировальный, 5, работа, ...",3,-
9,работа фрезерной машинкой,1,0,0,работа фрезерный машинка,1,"машинка, фрезерный, копировальный, 5, работа, ...",3,-


### Определим является ли запрос информационным

In [53]:
df['Информационный_характер'] = df['Фраза'].str.split()

In [54]:
def find_questions(splitted_phrase):
    for word in list_of_questions['vopros'].str.lower():
        if word in splitted_phrase:
            return 'инфо'

In [55]:
df['Информационный_характер'] = df['Информационный_характер'].apply(find_questions)

In [56]:
df['Информационный_характер'].fillna(value='-', inplace=True)

In [57]:
df

,Фраза,Базовая частота,Частота **,Частота *!*,Леммы,Кластер,Название_кластера,Длина_фразы_без_предлогов,Город,Информационный_характер
0,сварочные работы за тонну,99,1,0,сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ...",3,-,-
1,сварочные работы за тонну металлоконструкций,19,1,1,сварочный работа за тонна металлоконструкция,0,"металлоконструкция, тонна, сварочный, металл, ...",4,-,-
2,стоимость сварочно монтажных работ трубопровод...,29,29,29,стоимость сварочный монтажный работа трубопров...,0,"металлоконструкция, тонна, сварочный, металл, ...",6,-,-
3,цена сварочных работ за тонну металла,20,20,20,цена сварочный работа за тонна металл,0,"металлоконструкция, тонна, сварочный, металл, ...",5,-,-
4,стоимость сварочных работ за тонну,71,28,28,стоимость сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ...",4,-,-
...,...,...,...,...,...,...,...,...,...,...
3075,термообработка металла 5,1,0,0,термообработка металл 5,613,"по, нагрев, температура, старение, спб, 5, цен...",3,-,-
3076,машиностроение и металлообработка в китае,2,0,0,машиностроение и металлообработка в китай,614,"машиностроение, китай, металлообработка",3,-,-
3077,металлообработка в китае,5,2,2,металлообработка в китай,614,"машиностроение, китай, металлообработка",2,-,-
3078,ищу исполнителя по металлообработке,2,0,0,искать исполнитель по металлообработка,615,"по, исполнитель, искать, металлообработка",4,-,-


### Определим является ли запрос "коммерческим"

In [58]:
df['Коммерческий_характер'] = df['Леммы'].str.split()

In [59]:
def find_commerce(splitted_lemmas):
    for word in list_of_commerce_words['komm'].str.lower():
        if word in splitted_lemmas:
            return 'коммерческий'

In [60]:
df['Коммерческий_характер'] = df['Коммерческий_характер'].apply(find_commerce)

In [61]:
df['Коммерческий_характер'].fillna(value='-', inplace=True)

In [62]:
df

,Фраза,Базовая частота,Частота **,Частота *!*,Леммы,Кластер,Название_кластера,Длина_фразы_без_предлогов,Город,Информационный_характер,Коммерческий_характер
0,сварочные работы за тонну,99,1,0,сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ...",3,-,-,-
1,сварочные работы за тонну металлоконструкций,19,1,1,сварочный работа за тонна металлоконструкция,0,"металлоконструкция, тонна, сварочный, металл, ...",4,-,-,-
2,стоимость сварочно монтажных работ трубопровод...,29,29,29,стоимость сварочный монтажный работа трубопров...,0,"металлоконструкция, тонна, сварочный, металл, ...",6,-,-,коммерческий
3,цена сварочных работ за тонну металла,20,20,20,цена сварочный работа за тонна металл,0,"металлоконструкция, тонна, сварочный, металл, ...",5,-,-,коммерческий
4,стоимость сварочных работ за тонну,71,28,28,стоимость сварочный работа за тонна,0,"металлоконструкция, тонна, сварочный, металл, ...",4,-,-,коммерческий
...,...,...,...,...,...,...,...,...,...,...,...
3075,термообработка металла 5,1,0,0,термообработка металл 5,613,"по, нагрев, температура, старение, спб, 5, цен...",3,-,-,-
3076,машиностроение и металлообработка в китае,2,0,0,машиностроение и металлообработка в китай,614,"машиностроение, китай, металлообработка",3,-,-,-
3077,металлообработка в китае,5,2,2,металлообработка в китай,614,"машиностроение, китай, металлообработка",2,-,-,-
3078,ищу исполнителя по металлообработке,2,0,0,искать исполнитель по металлообработка,615,"по, исполнитель, искать, металлообработка",4,-,-,-


## Создание конечной готовой таблицы

Фразы с региональной принадлежностью определим в отдельный датафрейм - все, что не "спб", "санкт-петербург" и "-":

In [63]:
df_ready_regions = df[
    (df['Город'] != 'спб') & 
    (df['Город'] != 'санкт-петербург') & 
    (df['Город'] != '-')
]

Приоритетный регион - или "спб" или "санкт-петербург" или "-"::

In [64]:
df_ready = df[
    (df['Город'] == 'спб') |
    (df['Город'] == 'санкт-петербург') | 
    (df['Город'] == '-')
]

## Экспорт

In [65]:
writer_kernel = pd.ExcelWriter('ready_table.xlsx', engine='xlsxwriter')
df_ready.to_excel(writer_kernel, sheet_name='Основной список запросов', index=False)
df_ready_regions.to_excel(writer_kernel, sheet_name='Регионы', index=False)
writer_kernel.save()